# Extract

In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np

# get html data first
html_data = requests.get("https://en.wikipedia.org/wiki/World_Happiness_Report")

# check if status is 200 -> shows that its allowed to scrape the webpage
print(html_data.status_code)

# parse html data now using BeautifulSoup
soup = BeautifulSoup(html_data.text, "html.parser")
# print(soup)

# get all tables from wikipedia page
tables = soup.find_all('table',{'class':"wikitable"})

# store target table
table = tables[2]
# print(table)

# convert table html code to pandas df
data = pd.read_html(str(table))
# print(data)
df_happiness = pd.DataFrame(data[0])
display(df_happiness)


200


,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.809,1.285,1.500,0.961,0.662,0.160,0.478
1,2,Denmark,7.646,1.327,1.503,0.979,0.665,0.243,0.495
2,3,Switzerland,7.560,1.391,1.472,1.041,0.629,0.269,0.408
3,4,Iceland,7.504,1.327,1.548,1.001,0.662,0.362,0.145
4,5,Norway,7.488,1.424,1.495,1.008,0.670,0.288,0.434
...,...,...,...,...,...,...,...,...,...
148,149,Central African Republic,3.476,0.041,0.000,0.000,0.293,0.254,0.028
149,150,Rwanda,3.312,0.343,0.523,0.572,0.604,0.236,0.486
150,151,Zimbabwe,3.299,0.426,1.048,0.375,0.377,0.151,0.081
151,152,South Sudan,2.817,0.289,0.553,0.209,0.066,0.210,0.111


In [3]:
from tqdm import tqdm

# rename some countries to later match the country names from RapidAPI
df_happiness = df_happiness.apply(lambda x: x.replace("Congo (Kinshasa)", "DR Congo"))
df_happiness = df_happiness.apply(lambda x: x.replace("Congo (Brazzaville)", "Congo"))
df_happiness = df_happiness.apply(lambda x: x.replace("Ivory Coast", "Côte d'Ivoire"))

# create URL and headers for API call
url = "https://world-population.p.rapidapi.com/allcountriesname"

# the headers can be found when logging in to your RapidAPI account and opening the link above
headers = {
    'x-rapidapi-host': "world-population.p.rapidapi.com",
    'x-rapidapi-key': "1ae0131274msh2c8337983915cbep16eba8jsnce5983341241"
}

# add population column first by setting all values to NaN
df_happiness["Population"] = np.nan

# loop over countries and get population
for country in tqdm(df_happiness["Country or region"].to_list()):
    # create querystring for API call
    querystring = {"country_name" : country}
    
    # create request and fetch response
    response = requests.request("GET", url, headers=headers, params=querystring)
    
    # add population to dataframe in case response is okay
    response_dict = json.loads(response.text)
    if response_dict["ok"] == True:
        population = response_dict["body"]["population"]
        df_happiness.loc[df_happiness["Country or region"] == country, "Population"] = population


  0%|                                                                                          | 0/137 [00:01<?, ?it/s]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:
# get html data first
html_data = requests.get("https://www.worlddata.info/average-age.php")

# check if status is 200 -> shows that its allowed to scrape the webpage
print(html_data.status_code)

# parse html data now using BeautifulSoup
soup = BeautifulSoup(html_data.text, "html.parser")

# get all tables from wikipedia page
tables = soup.find_all('table',{'class':"std100 hover"})

# store target table
table = tables[0]

# convert table html code to pandas df
data = pd.read_html(str(table))
df_average_age = pd.DataFrame(data[0]) 
df_average_age

200


,Country,Median age in years,Population under 20 years old,Life expectancy in years
0,Japan,48.6,16.3 %,84.6
1,Germany,47.8,18.5 %,81.0
2,Italy,46.5,17.5 %,82.9
3,Hong Kong *,45.6,15.7 %,85.7
4,Greece,45.3,19.3 %,80.3
...,...,...,...,...
122,Chad,16.1,58.4 %,52.6
123,Mali,16.0,58.5 %,59.0
124,Angola,15.9,55.6 %,61.7
125,Uganda,15.7,57.1 %,62.7


In [5]:

df_final = df_happiness.set_index("Country or region").join(df_average_age.set_index("Country")).reset_index()

In [6]:
# Compute GDP by using GDP per capita and the Population columns
df_final["GDP"] = df_final["GDP per capita"] * df_final["Population"]

# Remove % sign of Population under 20 years old column and convert it to be of type float
def transform_col(col_val):
    try: 
        return float(col_val.replace(" %", ""))
    except: # value is NaN
        return col_val

df_final["Population under 20 years old in %"] = df_final["Population under20 years old"].apply(transform_col)
df_final = df_final.drop(columns=["Population under20 years old"])

KeyError: 'GDP per capita'

In [7]:
def load(dataset):
    dataset.to_csv("final_dataset.csv", index=False)

load(df_final)